In [1]:
# The goal is to deploy the same application into a multinode and a single node vespa cluster with all schemas
# but when deployed to multi-node the two schemas should be deployed to two separate content clusters
# while in single node deploynmetn both schemas live in the same content clusters.
# Also, let's use the single `hosts.xml` file.
# Use preprocess:include directive.
# TODO: have two content clusters with separated schemas

# The single node should be started as a separate docker container.
# Two techniques of variants for container and content clusters
## There is the `include` tag https://docs.vespa.ai/en/reference/applications/services/container.html#include
##

In [27]:
%%bash
docker run \
  --detach \
  --rm \
  --name vespa-demo-dev \
  --hostname vespa-demo-dev \
  --publish 0.0.0.0:8089:8080 \
  --publish 0.0.0.0:19059:19050 \
  --publish 0.0.0.0:19079:19071 \
  vespaengine/vespa:8.625.17

2193fb91ce264717e53bc4745e28c1188967c48484b5d957cfb7b20731b5a5b3


In [47]:
!zip -r application.zip .

updating: hosts.xml (deflated 70%)
updating: services.xml (deflated 73%)
updating: schemas/ (stored 0%)
updating: schemas/doc2.sd (deflated 34%)
updating: schemas/doc1.sd (deflated 33%)
updating: docker-compose.yml (deflated 80%)
updating: session.ipynb (deflated 98%)
updating: content-doc2.xml (deflated 58%)


In [55]:
%%bash
# Deploy to the vespa-demo-dev container as `instance=dev`
curl -X POST \
        --header "Content-Type:application/zip" \
                 --data-binary @application.zip \
    "http://localhost:19079/application/v2/tenant/default/prepareandactivate?instance=dev"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8651    0   318  100  8333    551  14457 --:--:-- --:--:-- --:--:-- 15019


{"log":[],"message":"Session 6 for tenant 'default' prepared and activated.","session-id":"6","activated":true,"tenant":"default","url":"http://localhost:19071/application/v2/tenant/default/application/default/environment/prod/region/default/instance/dev","configChangeActions":{"restart":[],"refeed":[],"reindex":[]}}

In [35]:
# Check which schemas are deployed to the instance=local-1
!docker exec vespa-demo-dev sh -c "ls /opt/vespa/var/db/vespa/search/cluster.content/n0/documents"

doc1
doc2


In [36]:
# Both indices as expected are in the node

In [26]:
!docker stop vespa-demo-dev

vespa-demo-dev


In [39]:
!docker compose -f docker-compose.yml up -d

[+] Building 0.0s (0/0)                                           docker:colima
[+] Running 1/0
 ✔ Network vespanet  Created                                               0.1s 
 ⠋ Container node1   Creating                                              0.0s 
 ⠋ Container node2   Creating                                              0.0s 
 ⠋ Container node0   Creating                                              0.0s 
[+] Running 4/4
 ✔ Network vespanet  Created                                               0.1s 
 ✔ Container node1   Created                                               0.1s 
 ✔ Container node2   Created                                               0.1s 
 ✔ Container node0   Created                                               0.1s 
 ⠋ Container node6   Creating                                              0.0s 
 ⠋ Container node3   Creating                                              0.0s 
 ⠋ Container node4   Creating                                              0.0

In [48]:
%%bash
# Deploy to the docker compose cluster as `instance=prod`
curl -X POST \
        --header "Content-Type:application/zip" \
                 --data-binary @application.zip \
    "http://localhost:19071/application/v2/tenant/default/prepareandactivate?instance=prod"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9009    0   676  100  8333    512   6936  0:00:01  0:00:01 --:--:--  6938  6323  0:00:01  0:00:01 --:--:--  6840


{"log":[{"time":1768677658532,"level":"INFO","message":"Coverage policy is 'group', but with 2 groups in the cluster all load will be placed on 1 group when the other group is allowed to be down when doing maintenance or upgrades. This might lead to overload. See https://docs.vespa.ai/en/reference/services-content.html#coverage-policy.","applicationPackage":true}],"message":"Session 5 for tenant 'default' prepared and activated.","session-id":"5","activated":true,"tenant":"default","url":"http://localhost:19071/application/v2/tenant/default/application/default/environment/prod/region/default/instance/prod","configChangeActions":{"restart":[],"refeed":[],"reindex":[]}}

In [52]:
!docker exec node7 sh -c "ls /opt/vespa/var/db/vespa/search/cluster.content-doc1/n1/documents"

doc1


In [ ]:
# ^ Has doc1 index, as expected

In [50]:
!docker exec node9 sh -c "ls /opt/vespa/var/db/vespa/search/cluster.content-doc2/n1/documents"

doc2


In [ ]:
# ^ Has doc2 index, as expected

In [54]:
# Is container deployed to 8080
!docker exec node4 sh -c "curl 0:8080/search"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   243    0   243    0     0   9720      0 --:--:-- --:--:-- --:--:--  9720
{"root":{"id":"toplevel","relevance":1.0,"fields":{"totalCount":0},"errors":[{"code":3,"summary":"Illegal query","source":"content-doc1","message":"No query"},{"code":3,"summary":"Illegal query","source":"content-doc2","message":"No query"}]}}

In [ ]:
# ^ node4 has /search endpoint activated